In [ ]:
import dotenv
import os
import sys

from pathlib import Path

In [ ]:
# Load and set environment

dotenv.load_dotenv()
os.environ['USER_AGENT'] = 'myagent'
PROJECT_HOME = Path(os.environ.get('PROJECT_HOME', Path.cwd() / '..')).resolve()
sys.path.append(str(PROJECT_HOME))

----

In [ ]:
# Connect to the vector DB as a retriever.
# This is the place to define the search algorithm, parameters and how many values (`k`) to return per query.

from app.databases.milvus import Milvus

vector_db = Milvus()
retriever = vector_db.as_retriever(search_kwargs={'k': 8})

In [ ]:
from langchain_aws import ChatBedrock
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import SystemMessage

# The ChatBot LLM
llm = ChatBedrock(
    model_id='anthropic.claude-3-5-sonnet-20240620-v1:0',
    region_name='us-east-1',
    model_kwargs=dict(temperature=0),
)

# Retriever tool, for the R in RAG.
tool = create_retriever_tool(
    retriever,
    'Internal_Company_Info_Retriever',
    'Searches and retrieves data from the corpus of documents that the company has',

    # Controls how the returned results will look when passed to the LLM.
    # To see available `variables`, check the `retriever.invoke('some query')[0].metadata.keys()`
    document_prompt=PromptTemplate(input_variables=['source', 'page_content'], template='\n'.join(['Source:', '{source}', 'Content:', '{page_content}'])),
    document_separator='\n\n========\n\n',
)
tools = [tool]

# Store conversation history. In prod, should replace with a real DB.
memory = SqliteSaver.from_conn_string(":memory:")

# The prompt message to use with the LLM. Provide clear and concise instructions to the LLM.
prompt_message = SystemMessage("""When answering the user question using data from the tools, be sure to:
1. First list the sources you are going to use in your answer. The list of sources should be of the form:
[1] - full/path/to/file (index of document in the retriever's answer (e.g. 0, 4): <The quote that being used>
[2] - ...
2. Add a separator '\n\n+++++++++++++\n\n'
3. Write a concise answer based only on the sources and quotes you listed above.

If you don't know the answer, answer that you don't know based on the information you have. If you're not sure, state that you're not sure.""")

# Create the agent itself.
agent_executor = create_react_agent(llm, tools, checkpointer=memory, state_modifier=prompt_message)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# To start a new conversation, change the `thread_id` to a new value.
config = {"configurable": {"thread_id": "user_12837"}}

while True:

    # Make a pretty separator for the next human message
    HumanMessage('').pretty_print()

    # Get user question.
    question = input('Write a question to the LLM. Write `q` or `quit` to exit:')

    # System output
    SystemMessage('').pretty_print()
    
    if question.lower() in ('q', 'quit'):
        # Quit
        print('Quitting for now. You can continue the conversation by rerunning this cell')
        break

    print('Processing question')
    
    # Ask the LLM.
    for s in agent_executor.stream({"messages": [HumanMessage(content=question)]}, config=config):
        if 'agent' in s and s['agent']['messages'][-1] and getattr(s['agent']['messages'][-1], 'tool_calls', None):
            print('Searching Vector DB for data')
        elif 'tools' in s:
            print('Analyzing retrieved data for answers')

    s['agent']['messages'][0].pretty_print()

----
#### Debugging

In [ ]:
# Let's you see more metadata about the conversation.
state = agent_executor.get_state(config)
state

----